# Problems of Chapter 4

## 4.2

In [ ]:
import tabulate


def make_arithmetic_table(name, elements, op):
    return tabulate.tabulate(
        [[str(op(a, b)) for a in elements] for b in elements],
        headers=[name, *map(str, elements)],
        showindex=map(str, elements),
        tablefmt="html",
    )

In [ ]:
m = 7
make_arithmetic_table("+", range(m), lambda a, b: (a + b) % m)

In [ ]:
make_arithmetic_table("*", range(m), lambda a, b: (a * b) % m)

## 4.3

In [ ]:
from polynomial import PolyGF2

p = PolyGF2(0b1011)
elements = [PolyGF2(x) for x in range(2**p.degree)]
make_arithmetic_table("*", elements, lambda a, b: (a * b) % p)

## 4.4

In [ ]:
p = PolyGF2(0b10011)

operands = [
    (PolyGF2(0b0101), PolyGF2(0b1101)),
    (PolyGF2(0b0101), PolyGF2(0b0011)),
]

for i, (a, x) in enumerate(operands, start=1):
    print(f"{i}. ({a}) + ({x}) = {a + x}")

## 4.6

In [ ]:
def reciprocal(a: PolyGF2, p: PolyGF2) -> PolyGF2:
    # brute-force search
    for b in (PolyGF2(x) for x in range(1, 2**p.degree)):
        if a * b % p == 1:
            return b
    raise ArithmeticError(f"cannot find inverse of {a!r} in GF(2^{p.degree})")


p = PolyGF2(0b1_0001_1011)
a = PolyGF2(0b0001_0011)
x = PolyGF2(0b1100_1100)

SUPERSCRIPT_MINUS_ONE = "\N{Superscript Minus}\N{Superscript One}"
print(f"({a})({x}){SUPERSCRIPT_MINUS_ONE} = {a * reciprocal(x, p) % p}")

## 4.8

In [ ]:
from collections.abc import Iterator


def find_irreducible_polys(degree: int) -> Iterator[PolyGF2]:
    candidates = [PolyGF2(x) for x in range(2**degree, 2 ** (degree + 1))]
    factors = [PolyGF2(x) for x in range(2, 2**degree)]

    for candidate in candidates:
        for factor in factors:
            if candidate % factor == 0:
                break
        else:
            yield candidate


# https://mathworld.wolfram.com/IrreduciblePolynomial.html

degrees = (3, 4)

tabulate.tabulate(
    [[", ".join(str(x) for x in find_irreducible_polys(m))] for m in degrees],
    headers=["m", "irreducible polynomials"],
    showindex=degrees,
    tablefmt="html",
)

## 4.14

In [ ]:
from aes.aes import S_BOX
from aes.utils import matrix_by_vector

# https://en.wikipedia.org/wiki/Rijndael_S-box#Forward_S-box

# def affine_transformation(x: PolyGF2) -> PolyGF2:
#     return x * 31 % 257 + 99

A = (
    (1, 0, 0, 0, 1, 1, 1, 1),
    (1, 1, 0, 0, 0, 1, 1, 1),
    (1, 1, 1, 0, 0, 0, 1, 1),
    (1, 1, 1, 1, 0, 0, 0, 1),
    (1, 1, 1, 1, 1, 0, 0, 0),
    (0, 1, 1, 1, 1, 1, 0, 0),
    (0, 0, 1, 1, 1, 1, 1, 0),
    (0, 0, 0, 1, 1, 1, 1, 1),
)

b = (1, 1, 0, 0, 0, 1, 1, 0)


def affine_transformation(x: PolyGF2) -> PolyGF2:
    return PolyGF2.from_vector(matrix_by_vector(A, x)) + PolyGF2.from_vector(b)


def aes_inverse(x: PolyGF2) -> PolyGF2:
    if x == 0:
        return x
    return reciprocal(x, PolyGF2(0b1_0001_1011))


given = (0x29, 0xF3, 0x01)

assert all(S_BOX[x] == affine_transformation(aes_inverse(PolyGF2(x))) for x in given)
assert S_BOX[0] == PolyGF2.from_vector(b)